In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [3]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [5]:
df_green.registerTempTable('green')

In [17]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1, 2
""")

In [18]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 00:00:00|   7| 769.7299999999998|            45|
|2020-01-01 00:00:00|  17|            195.03|             9|
|2020-01-01 00:00:00|  18|               7.8|             1|
|2020-01-01 00:00:00|  22|              15.8|             1|
|2020-01-01 00:00:00|  24|              87.6|             3|
|2020-01-01 00:00:00|  25|             531.0|            26|
|2020-01-01 00:00:00|  29|              61.3|             1|
|2020-01-01 00:00:00|  32| 68.94999999999999|             2|
|2020-01-01 00:00:00|  33|            317.27|            11|
|2020-01-01 00:00:00|  35|129.95999999999998|             5|
|2020-01-01 00:00:00|  36|            295.34|            11|
|2020-01-01 00:00:00|  37|            175.67|             6|
|2020-01-01 00:00:00|  38| 98.78999999999999|             2|
|2020-01-01 00:00:00|  4

In [27]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [20]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [21]:
df_yellow.registerTempTable('yellow')

In [25]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1, 2
""")

In [26]:
df_yellow_revenue.write.parquet('data/report/revenue/yellow')

In [30]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [31]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [32]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 04:00:00| 127|             42.72|                   3|60.480000000000004|                    3|
|2020-01-01 05:00:00| 114|              null|                null|            720.75|                   43|
|2020-01-01 06:00:00|  65|             18.05|                   1|38.260000000000005|                    4|
|2020-01-01 06:00:00| 161|              null|                null|1513.7399999999998|                   43|
|2020-01-01 08:00:00|  17|              null|                null|             46.34|                    3|
|2020-01-01 08:00:00| 107|              null|                null| 791.9599999999998|                   48|
|2020-01-01 08:00:00| 162|  

In [33]:
df_join.write.parquet('data/report/revenue/total')

In [34]:
df_join = spark.read.parquet('data/report/revenue/total')

In [35]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 02:00:00|  42|            469.12|                  33| 950.1199999999997|                   73|
|2020-01-01 06:00:00| 228|              15.3|                   1|              null|                 null|
|2020-01-01 06:00:00| 246|              null|                null|            437.72|                   21|
|2020-01-01 09:00:00| 163|              null|                null|1990.4399999999991|                   94|
|2020-01-01 10:00:00|  66|             16.05|                   1|             66.22|                    4|
|2020-01-01 10:00:00| 114|              null|                null|293.45000000000005|                   19|
|2020-01-01 12:00:00| 123|  